In [1]:
import sys
!{sys.executable} -m pip install openpifpaf
!{sys.executable} -m pip install matplotlib

from IPython.display import Image, display

#             print("image with ID {} has {} bboxes".format(imageId, openpifpaf_prediction))          
#             display(Image(filename=image_path))            

In [2]:
%matplotlib inline

import io
import numpy as np
import openpifpaf
import PIL
import requests
import torch

device = torch.device('cpu')
# device = torch.device('cuda')  # if cuda is available

print(openpifpaf.__version__)
print(torch.__version__)

# image_response = requests.get('https://raw.githubusercontent.com/vita-epfl/openpifpaf/master/docs/coco/000000081988.jpg')

# Prediction API set up
net_cpu, _ = openpifpaf.network.factory(checkpoint='shufflenetv2k16w', download_progress=False)
net = net_cpu.to(device)

openpifpaf.decoder.CifSeeds.threshold = 0.5
openpifpaf.decoder.nms.Keypoints.keypoint_threshold = 0.2
openpifpaf.decoder.nms.Keypoints.instance_threshold = 0.2
processor = openpifpaf.decoder.factory_decode(net.head_nets, basenet_stride=net.base_net.stride)

preprocess = openpifpaf.transforms.Compose([
  openpifpaf.transforms.NormalizeAnnotations(),
  openpifpaf.transforms.CenterPadTight(16),
  openpifpaf.transforms.EVAL_TRANSFORM,
])

keypoint_painter = openpifpaf.show.KeypointPainter(color_connections=True, linewidth=6)

def get_number_bbox(image_path):
    pil_im = PIL.Image.open(image_path).convert('RGB')
    im = np.asarray(pil_im)
    
    data = openpifpaf.datasets.PilImageList([pil_im], preprocess=preprocess)

    loader = torch.utils.data.DataLoader(
      data, batch_size=1, pin_memory=True, 
      collate_fn=openpifpaf.datasets.collate_images_anns_meta)
    
    for images_batch, _, __ in loader:
        predictions = processor.batch(net, images_batch, device=device)[0]
    
    return len(predictions), im, predictions

0.11.9
1.7.0


In [ ]:
import openpifpaf
import json
import os.path

f = open('results.csv','w')
print('ImageID,PifPaf Prediction,Total bbox', file=f)

with open('nightowls_validation.json') as json_file:
    data = json.load(json_file)
    
    number_of_bbox_of_current_image = 0
    
    # Final variables
    n_bbox_found = 0
    n_bbox_ground_truth = 0
    
    previous_image_id = 7000000
    previous_image_path = 'nightowls_validation/58c5812cbc260137e096966f.png'
    
    for annotation in data["annotations"]:
        current_image_id = annotation['image_id']
        image_file_name = data['images'][current_image_id - 7000000]['file_name']
        current_image_path = 'nightowls_validation/' + image_file_name
    
        # Process previous image
        if current_image_path != previous_image_path:
            print("OK, we finished with image: ", previous_image_id)
            openpifpaf_prediction, im, predictions = get_number_bbox(previous_image_path)
            # quanti c'erano - quanti pifpaf ha trovato = missed annoations
#             missed_bbox = max(0, number_of_bbox_of_current_image - openpifpaf_prediction)
#             n_bbox_missed += missed_bbox
            n_bbox_found += openpifpaf_prediction
    
#             with openpifpaf.show.image_canvas(im) as ax:
#                 keypoint_painter.annotations(ax, predictions)
            
            print('{},{},{}'.format(previous_image_id, openpifpaf_prediction, number_of_bbox_of_current_image), file=f)
    
            # Reset
            previous_image_path = current_image_path
            previous_image_id = current_image_id
            number_of_bbox_of_current_image = 0                            
        
        # Process current image
        if os.path.isfile(current_image_path) and annotation['ignore'] == 0:
            number_of_bbox_of_current_image += 1
            n_bbox_ground_truth += 1
            
            # risultato finale = # missed / # totali
            print("Image ID: {}, total found: {}, total bbox: {}".format(current_image_id, n_bbox_found, n_bbox_ground_truth))
        
        if n_bbox_ground_truth == 2000:
            break
    
    f.close()
    print("Done! Average miss rate: {}".format(str(float(n_bbox_ground_truth - n_bbox_found) / float(n_bbox_ground_truth))))


Image ID: 7000000, total found: 0, total bbox: 1
Image ID: 7000000, total found: 0, total bbox: 2
OK, we finished with image:  7000000
Image ID: 7000001, total found: 0, total bbox: 3
Image ID: 7000001, total found: 0, total bbox: 4
OK, we finished with image:  7000001
Image ID: 7000002, total found: 0, total bbox: 5
Image ID: 7000002, total found: 0, total bbox: 6
OK, we finished with image:  7000002
Image ID: 7000003, total found: 0, total bbox: 7
Image ID: 7000003, total found: 0, total bbox: 8
OK, we finished with image:  7000003
Image ID: 7000004, total found: 0, total bbox: 9
Image ID: 7000004, total found: 0, total bbox: 10
OK, we finished with image:  7000004
Image ID: 7000005, total found: 0, total bbox: 11
Image ID: 7000005, total found: 0, total bbox: 12
OK, we finished with image:  7000005
Image ID: 7000006, total found: 0, total bbox: 13
Image ID: 7000006, total found: 0, total bbox: 14
OK, we finished with image:  7000006
Image ID: 7000007, total found: 0, total bbox: 15


In [12]:
import numpy as np
import pandas as pd

pif_night_data = './results.csv'
data = pd.read_csv(pif_night_data)
pifpaf = data['PifPaf Prediction']
ground_truth = data['Total bbox']
total_MR = 0
for i in range(len(pifpaf)):
    #If pifpaf detects more we should discard the value
    #MR of image x 
    if(ground_truth[i] != 0 and pifpaf[i]<=ground_truth[i]):
        MR = max(0,ground_truth[i]-pifpaf[i])/ground_truth[i]
        #print('MR of image id '+str(data['ImageID'][i])+' is '+str(MR))
        total_MR +=MR

#Average Miss rate 
aMR = total_MR/len(pifpaf)
print("Average miss rate = "+str(aMR))
    

Average miss rate = 0.6861519607843136
